# Normal_1 Full Quant Pipeline (Step1~Step5)

这个 notebook 调用 `scripts/run_normal1_pipeline.py`，一键跑通总流程并汇总指标。

In [ ]:
from pathlib import Path

cwd = Path.cwd()
if (cwd / 'scripts').exists():
    repo = cwd
elif (cwd.parent / 'scripts').exists():
    repo = cwd.parent
else:
    raise RuntimeError('Cannot locate repo root containing scripts/')

ct = repo / 'ASOCA2020/Normal/CTCA_nii/Normal_1.nii.gz'
gt_mask = repo / 'ASOCA2020/Normal/Annotations_nii/Normal_1.nii.gz'
gt_centerline = repo / 'ASOCA2020/Normal/Centerlines/Normal_1.vtp'
out_dir = repo / 'outputs/quant/Normal_1'

step2_backend = 'vmtk'
thr_mm = 1.0
skip_existing = True

print('repo:', repo)
print('out_dir:', out_dir)

In [ ]:
import subprocess
import sys

cmd = [
    sys.executable,
    str(repo / 'scripts/run_normal1_pipeline.py'),
    '--ct', str(ct),
    '--gt-mask', str(gt_mask),
    '--gt-centerline', str(gt_centerline),
    '--out-dir', str(out_dir),
    '--step2-backend', step2_backend,
    '--thr', str(thr_mm),
]

if skip_existing:
    cmd.append('--skip-existing')

print('Running command:
', ' '.join(cmd))
result = subprocess.run(cmd, cwd=repo)
print('exit code:', result.returncode)
if result.returncode != 0:
    raise RuntimeError('Pipeline failed; check logs above.')

In [ ]:
import json

summary_path = out_dir / 'pipeline_run_summary.json'
if not summary_path.exists():
    raise FileNotFoundError(summary_path)

summary = json.loads(summary_path.read_text(encoding='utf-8'))

rows = []
for stage_name in ('step1', 'step2', 'step3', 'step4', 'step5'):
    stage = summary.get('stages', {}).get(stage_name)
    if not stage:
        rows.append({'stage': stage_name, 'status': 'missing'})
        continue
    row = {'stage': stage_name, 'status': 'ok'}
    metrics = stage.get('metrics', {})
    row.update(metrics)
    rows.append(row)

try:
    import pandas as pd
    display(pd.DataFrame(rows))
except Exception:
    for row in rows:
        print(row)